In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import json
import tqdm
import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score

from model import ModelType
from dataset import DatasetType
from utils import *


RUN_NAME = "20240703_183141"
DATA_ROOT = ""
BATCH_SIZE = 32
DEVICE = "cuda"


In [ ]:
# load given inline arguments
with open(os.path.join(".", "runs", RUN_NAME, "args.json")) as f:
    run_args = json.loads(f.read())

if DATA_ROOT.strip() == "":
    DATA_ROOT = run_args["dataset_root"]

assert os.path.exists(DATA_ROOT), "Invalid dataset root path!"

In [ ]:
# load model
model = ModelType()
model = model.to(DEVICE)
_ = load_checkpoint(
    model=model,
    device=DEVICE,
    dirpath=os.path.join(".", "runs", RUN_NAME),
    fname="best_chkpt.pt",
)
num_params = count_model_params(model)
print("The model has {:,} parameters.".format(num_params))

In [ ]:
# load dataset
ds = DatasetType(DATA_ROOT, "test")
data_loader = torch.utils.data.DataLoader(dataset=ds, batch_size=BATCH_SIZE, shuffle=True)
print("Testing dataset has {:,} samples.".format(len(ds)))

In [ ]:
# example test loop
model.eval()

predictions = []
groud_truth = []

for inputs, targets in tqdm(data_loader, desc="Evaluation"):
    inputs = inputs.to(DEVICE)
    targets = targets.to(DEVICE)

    outputs = model(inputs)

    predictions.append(outputs.detach().cpu().numpy())
    groud_truth.append(targets.detach().cpu().numpy())

# assuming the first axis is the batch size
predictions = np.concatenate(predictions, axis=0)
groud_truth = np.concatenate(groud_truth, axis=0)
